# Import libraries

In [0]:
import json

import pandas as pd
import numpy as np

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from tqdm import tqdm_notebook

⚠️⚠️⚠️

**You can jump to the end of preprocessing if you use the .pkl file.**

# Loading data

In [0]:
# loading data from JSONs located in Drive

from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/data/" # path may need to be adjusted

reviews_luxury = [json.loads(line) for line in open(path+'Luxury_Beauty_5.json', 'r')]
reviews_software = [json.loads(line) for line in open(path+'Software_5.json', 'r')]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# defining a function which extracts the text of the review from each JSON

def create_text(reviews):
    """Takes as input a list containing JSONs and returns a set (to eliminate eventual doublets), in which each element is the text of one review"""
    text = set()
    for item in reviews:
        try:
            text.add(item['reviewText'])
        except KeyError:
            print("JSON badly formatted")
    return text

# applying create_text function to list of reviews keeping only 10k each

luxury_text, software_text = list(create_text(reviews_luxury))[:10000], list(create_text(reviews_software))[:10000]

JSON bad formatted
JSON bad formatted
JSON bad formatted
JSON bad formatted
JSON bad formatted
JSON bad formatted
JSON bad formatted
JSON bad formatted
JSON bad formatted
JSON bad formatted
JSON bad formatted
JSON bad formatted
JSON bad formatted
JSON bad formatted


In [0]:
# creating dataframe with reviews' text and labels

df = pd.DataFrame(luxury_text+software_text, columns = ['text'])
df["label"] = [0]*10000 + [1]*10000
df.head()

,text,label
0,This lotion is sticky and I'm not crazy about ...,0
1,The smell of this cologne is fresh and light! ...,0
2,"When getting this product, I couldn't tell whe...",0
3,I must agree with the majority of reviewers: ...,0
4,So far I realky like this set. The setting po...,0


# Preprocessing 

In [0]:
# text pre-processing where a clean (lemmatized, without stopwords, punctuation, or symbols) dataset is created (takes some time to process everything -more than 10 mins in colab- but you can directly use the pickle we provide)

text = df["text"].tolist() # extract the texts from dataframe

stop = list(STOP_WORDS)+["\n", "this"]
more_punct = list(punctuation)

nlp = spacy.load("en") # initialize spaCy

def clean_component(doc):
    """Takes a string (one review) and outputs a clean pre-processed text"""
    doc = nlp(doc)
    doc = [tok.lemma_ for tok in doc if (tok.text not in stop and tok.text not in more_punct and tok.pos_ != "PUNCT" and tok.pos_ != "SYM" and tok.pos_ != "PRON")] # lemmitize and cleans unwanted data
    doc = ' '.join(doc)
    doc = doc.replace("\n\n", " ").replace("\n", " ").replace("-PRON-", "").replace("this", "") # further cleaning
    return doc

clean_sent = [clean_component(sentence) for sentence in text] # applying clean_component to the list with all the reviews

In [0]:
# updating dataframe and taking X and y as data

df["clean_text"] = clean_sent
X = df["clean_text"]
y = df["label"]
df.head()

,text,label,clean_text
0,This lotion is sticky and I'm not crazy about ...,0,lotion sticky crazy scent imagine good sunt...
1,The smell of this cologne is fresh and light! ...,0,the smell cologne fresh light perfect man life...
2,"When getting this product, I couldn't tell whe...",0,when get product tell skin tone cool warm tint...
3,I must agree with the majority of reviewers: ...,0,agree majority reviewer product effective ...
4,So far I realky like this set. The setting po...,0,so far realky like set the set powder transl...


In [0]:
# Saving the clean data locally
df.to_pickle(path+'dataframe.pkl')  # where to save it, usually as a .pkl

In [0]:
# To read the df next time 
# df = pd.read_pickle("dataframe.pkl")

# Data preparation

## Shaping the data

In [0]:
# train, unlabel, test shuffling and splitting. 
# We pretend that a part of the data (X_unlabel) does not have the labels. 
# So, we will never use y_unlabel

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42) # 20% of data for test
X_train, X_unlabel, y_train, y_unlabel = train_test_split(X_train, y_train, test_size=0.60, random_state=42) # 60% of training data will be unlabled

print(X_train.shape)
print(X_unlabel.shape)
print(X_test.shape)

(6400,)
(9600,)
(4000,)


## Feature engineering

In [0]:
# vectorize label, unlabel, test sets using TF-IDF technique

# to uncomment the first time you run this cell, if you use colab
nltk.download('punkt') 

# we keep a max of 9k features,
# we do not lowercase since in the data we have many names of products, 
# we further remove stopwords and use up to 3-grams
tfidf_vectorizer = TfidfVectorizer(max_features=9000,
                                  lowercase=False,
                                  use_idf=True,
                                  stop_words="english",
                                  tokenizer=nltk.word_tokenize,
                                  ngram_range=(1, 3))

X_label = tfidf_vectorizer.fit_transform(X_train).toarray()
X_unlabel = tfidf_vectorizer.fit_transform(X_unlabel).toarray()
X_test = tfidf_vectorizer.fit_transform(X_test).toarray()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Machine learning 

## Choice of the model

In [0]:
# baseline estimation with different classifiers and cross validation 
# to see which classifier works better

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=42),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(solver='lbfgs', multi_class='auto'),
]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []

for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model,
                                X_label,
                                y_train,
                                scoring='accuracy',
                                cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

cv_df.groupby('model_name').accuracy.mean()

model_name
LinearSVC                 0.972500
LogisticRegression        0.975937
MultinomialNB             0.976094
RandomForestClassifier    0.898125
Name: accuracy, dtype: float64

In [0]:
# Normal submission with the highest model predicted by the baseline. 
# This same classifirìer will be used in the pseudo-labelling part and results will be compared

nb = MultinomialNB()
nb.fit(X_label, y_train)
pred = nb.predict(X_test)
accuracy_score(pred, y_test)

0.72025

## Pseudo-labeller implementation

In [0]:
# defining the class which will handle the pseudo-labelling

class pseudo_labeling():

    
    def __init__(self,
                 model,
                 unlabelled_data,
                 sample_rate=0.01,
                 upper_threshold=0.6,
                 lower_threshold=0.4,
                 verbose=False):

        # defining initial parameteres 

        # percentage of unlabeled data taken in each iteration
        self.sample_rate = sample_rate 
        # ML model to give to the pseudo-labeler
        self.model = model 
        self.unlabelled_data = unlabelled_data
        self.verbose = verbose
        self.upper_threshold = upper_threshold
        self.lower_threshold = lower_threshold
        
        # create a list of all the indices of the unlabeled samples
        self.unlabelled_indices = list(range(unlabelled_data.shape[0]))
        
        # number of rows from unlabeled data to sample in each iteration 
        # (based on the sample rate given as parameter)
        self.sample_size = int(unlabelled_data.shape[0] * self.sample_rate)
        
        # shuffle the list of indices
        np.random.shuffle(self.unlabelled_indices)

        
    
    def __pop_rows(self):
        """
        Function to sample indices without replacement
        """
        # takes n rows (based on sample_size) from the shuffled list of indices
        chosen_rows = self.unlabelled_indices[:self.sample_size] 
        
        # remove the chosen rows from the list of indices 
        # (we are sampling without replacement)
        self.unlabelled_indices = self.unlabelled_indices[self.sample_size:]
        return chosen_rows
    
    
    def fit(self, X, y):
        
        """
        Perform pseudo labelling
        
        X: train labeled features
        y: train targets
        
        """
        
        # define number of iterations you will have
        # based on the number of unlabeled data and the sample size
        num_iters = int(len(self.unlabelled_indices)/self.sample_size) 

        for _ in (tqdm_notebook(range(num_iters)) if self.verbose else range(num_iters)):
            
            # get the samples
            chosen_rows = self.__pop_rows()

            # fit to data
            self.model.fit(X, y.ravel())
            
            # takes unlabled rows
            # and makes the model predict over these unlabeled samples
            chosen_unlabelled_rows = self.unlabelled_data[chosen_rows,:]
            pseudo_labels_prob = self.model.predict_proba(chosen_unlabelled_rows)
            
            
            # we have 2 classes, this means predict_proba
            # returns an array of 2 probabilities per datapoint

            # we will first find the maximum probability,
            # and then find the rows which are within our threshold values
            label_probability = np.max(pseudo_labels_prob, axis = 1)

            # default values are 0,4 (for lower_threshold)
            # and 0,6 (for upper_threshold), 
            # but we will try all the combinations to find the best one
            labels_within_threshold = np.where((label_probability < self.lower_threshold) | (label_probability > self.upper_threshold))[0]
            
            
            # use argmax to find the class with the highest probability
            pseudo_labels = np.argmax(pseudo_labels_prob[labels_within_threshold], axis = 1)
            chosen_unlabelled_rows = chosen_unlabelled_rows[labels_within_threshold]

            # combine data 
            # (part of the pseudo-labeler where the already labeled data 
            # and the new ones, labled in the current iteration,
            # get combined to create new dataset for the new iteration)
            X = np.vstack((chosen_unlabelled_rows, X))
            y = np.vstack((pseudo_labels.reshape(-1,1), np.array(y).reshape(-1,1)))

            # shuffling
            indices = list(range(X.shape[0]))
            np.random.shuffle(indices)

            X = X[indices]
            y = y[indices]     
        
    def predict(self, X):
        return self.model.predict(X)
    
    def predict_proba(self, X):
        return self.model.predict_proba(X)
    
     
    def decision_function(self, X):
        return self.model.decision_function(X)

## Choice of the best parameters

In [0]:
# applying pseudo-labeling trying different parameters 
# to find the best combination (takes long time, around 50 mins in colab. A pickle is provided, though)

# initialization
result_df = pd.DataFrame(columns=["sample_rate","lower_threshold","upper_threshold","accuracy"])

# loop

# percentage of unlabled data given to the model for each iteration 
# (from 0.01 to 0.5)
sample_rate = 0.01 
while sample_rate < 0.5:
  lower_threshold = 0 # from 0 to 0.5
  while lower_threshold < 0.5:
    upper_threshold = 0.5 # from 0.5 to 1 
    while upper_threshold < 1:
      pseudo_labeller = pseudo_labeling(
        nb, # using the model previously used for normal submission to compare the results
        X_unlabel, # feed the pseudo-labeler with unlabeled data
        sample_rate = sample_rate,
        upper_threshold = upper_threshold,
        lower_threshold = lower_threshold,
        verbose = True
      )

      # fit the model
      pseudo_labeller.fit(X_label, y_train)
      y_test_pred = pseudo_labeller.predict(X_test)
      score = accuracy_score(y_test_pred, y_test)
      
      # dataframe for results
      result_df = result_df.append({"sample_rate":sample_rate,
                        "lower_threshold":lower_threshold,
                        "upper_threshold":upper_threshold,
                        "accuracy":score},ignore_index=True)
      upper_threshold = upper_threshold + 0.1
    lower_threshold = lower_threshold + 0.1
  sample_rate = sample_rate + 0.05


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [0]:
# to save the results locally
result_df.to_pickle(path+'best_parameter.pkl')

## Results

In [0]:
# maximal accuracy among all the trials
result_df.accuracy.max()

0.76175

In [0]:
# to see which combination is the best
result_df.loc[result_df['accuracy'].idxmax()]

sample_rate        0.16000
lower_threshold    0.20000
upper_threshold    0.50000
accuracy           0.76175
Name: 102, dtype: float64